# Install necessary libraries

In [1]:
!pip install transformers datasets peft torch python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from transformers import TrainingArguments, Trainer
import gc
from docx import Document

# Remove any pre-existing files from previous runs

In [3]:
os.remove('/content/fine-tuned-QA-tinyllama-1.1B') if os.path.exists('/content/fine-tuned-QA-tinyllama-1.1B') else None
os.remove('/content/outputs') if os.path.exists('/content/outputs') else None

# Define the model reference

In [4]:
model_ref = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Create the dataset with 30 riddles and solutions

In [5]:
qa_data = [
    {"question": "What number becomes zero when you subtract 15 from half of it?", "answer": "30"},
    {"question": "What three positive numbers give the same answer when multiplied and added together?", "answer": "1, 2, and 3"},
    {"question": "I am a number. Divide me by 3, then add 5, and you get 11. What number am I?", "answer": "18"},
    {"question": "I am a number less than 50. If you double me and subtract 10, you get 30. What number am I?", "answer": "20"},
    {"question": "A farmer has 17 sheep, and all but 9 run away. How many are left?", "answer": "9"},
    {"question": "The sum of two numbers is 16, and their difference is 4. What are the numbers?", "answer": "10 and 6"},
    {"question": "Double a number and subtract 8 to get 10. What number is it?", "answer": "9"},
    {"question": "A clock shows 3:15. What is the angle between the hour and the minute hand?", "answer": "7.5 degrees"},
    {"question": "What number do you get if you divide 30 by half and add 10?", "answer": "70"},
    {"question": "Take me and double my value. Add 8, divide by 4, and you get 5. What number am I?", "answer": "6"},
    {"question": "A box contains 12 apples. You take away 5. How many do you have?", "answer": "5 (You took them!)"},
    {"question": "If five cats catch five mice in five minutes, how many cats are needed to catch 100 mice in 100 minutes?", "answer": "5 cats"},
    {"question": "The sum of three consecutive even numbers is 54. What are the numbers?", "answer": "16, 18, 20"},
    {"question": "I am a two-digit number. My tens digit is 3 more than my ones digit. What number am I?", "answer": "41"},
    {"question": "I am thinking of a number. When I add 10 to it and then divide by 2, I get 12. What number am I?", "answer": "14"},
    {"question": "What is the smallest positive integer that is divisible by both 3 and 5?", "answer": "15"},
    {"question": "I am an odd number. Take away one letter, and I become even. What number am I?", "answer": "Seven"},
    {"question": "If two’s company and three’s a crowd, what are four and five?", "answer": "Nine"},
    {"question": "What is the sum of all angles in a triangle?", "answer": "180 degrees"},
    {"question": "How many faces does a cube have?", "answer": "6"},
    {"question": "If a book has 500 pages, how many pages would be left if I read 100 pages each day?", "answer": "400 pages"},
    {"question": "What is the square root of 64?", "answer": "8"},
    {"question": "How many sides does a hexagon have?", "answer": "6"},
    {"question": "What is the next number in the sequence: 2, 4, 8, 16?", "answer": "32"},
    {"question": "If you have 2 apples and I give you 3 more, how many apples do you have?", "answer": "5 apples"},
    {"question": "What is the area of a rectangle with length 5 and width 3?", "answer": "15 square units"},
    {"question": "How many seconds are in 1 minute?", "answer": "60"},
    {"question": "What is the sum of 7, 15, and 23?", "answer": "45"},
    {"question": "What is the product of 6 and 9?", "answer": "54"},
    {"question": "How many months are there in a year?", "answer": "12"},
    {"question": "What is the time after 6 hours from 2:00 PM?", "answer": "8:00 PM"},
    {"question": "If I am 6 years older than my sister and my sister is 10, how old am I?", "answer": "16"},
    {"question": "What is the perimeter of a square with side length 4?", "answer": "16 units"},
]

# Define the model and tokenizer

In [6]:
model_name = model_ref
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]


# Move model to GPU if available

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Format data for the fine-tuning task

In [8]:
def format_qa(example):
    return {"text": f"Question: {example['question']} Answer: {example['answer']}"}

qa_dataset = Dataset.from_list(qa_data)
formatted_dataset = qa_dataset.map(format_qa)

Map:   0%|          | 0/33 [00:00<?, ? examples/s]


# Tokenization

In [9]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

# Apply tokenization

In [10]:
tokenized_qa_dataset = formatted_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

# LoRA configuration for efficient fine-tuning

In [11]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

# Apply LoRA to the model

In [12]:
model = get_peft_model(model, lora_config)

# Training arguments

In [13]:
training_args = TrainingArguments(
    per_device_train_batch_size=5,
    gradient_accumulation_steps=3,
    warmup_steps=100,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    output_dir="outputs",
    report_to="none",
    remove_unused_columns=False,
)

# Initialize the trainer

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_qa_dataset,
)

# Clear GPU memory before training

In [15]:
gc.collect()
torch.cuda.empty_cache()

# Start training

In [16]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=6, training_loss=3.857762654622396, metrics={'train_runtime': 12.8932, 'train_samples_per_second': 7.678, 'train_steps_per_second': 0.465, 'total_flos': 210206214586368.0, 'train_loss': 3.857762654622396, 'epoch': 2.0})

# Save the fine-tuned model

In [17]:
model.save_pretrained("fine-tuned-QA-tinyllama-1.1B")
tokenizer.save_pretrained("fine-tuned-QA-tinyllama-1.1B")

('fine-tuned-QA-tinyllama-1.1B/tokenizer_config.json',
 'fine-tuned-QA-tinyllama-1.1B/special_tokens_map.json',
 'fine-tuned-QA-tinyllama-1.1B/tokenizer.model',
 'fine-tuned-QA-tinyllama-1.1B/added_tokens.json',
 'fine-tuned-QA-tinyllama-1.1B/tokenizer.json')

# Load the fine-tuned model

In [18]:
model_path = "fine-tuned-QA-tinyllama-1.1B"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=256, b

# Generate answers for test riddles

In [19]:
def generate_answer(question, max_length=50):
    prompt = f"Question: {question} Answer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, temperature=0.7, top_k=50, top_p=0.9)

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test case - Generate answers for some riddles

In [20]:
test_questions = [
    "What number becomes zero when you subtract 15 from half of it?",
    "The sum of two numbers is 16, and their difference is 4. What are the numbers?",
    "A farmer has 17 sheep, and all but 9 run away. How many are left?",
    "What is the square root of 64?",
    "How many faces does a cube have?"
]

for q in test_questions:
    print(f"Q: {q}")
    print(f"A: {generate_answer(q)}\n")

Q: What number becomes zero when you subtract 15 from half of it?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


A: Question: What number becomes zero when you subtract 15 from half of it? Answer: 15 becomes 0.

Q: The sum of two numbers is 16, and their difference is 4. What are the numbers?
A: Question: The sum of two numbers is 16, and their difference is 4. What are the numbers? Answer: The numbers are 16 and 12.

Q: A farmer has 17 sheep, and all but 9 run away. How many are left?
A: Question: A farmer has 17 sheep, and all but 9 run away. How many are left? Answer: 8

Q: What is the square root of 64?
A: Question: What is the square root of 64? Answer: The square root of 64 is 8.

Q: How many faces does a cube have?
A: Question: How many faces does a cube have? Answer: A cube has 6 faces.



# Save the 3 best riddles and solutions to a Python file

In [21]:
best_riddles = [
    {"question": "What number becomes zero when you subtract 15 from half of it?", "answer": "30"},
    {"question": "The sum of two numbers is 16, and their difference is 4. What are the numbers?", "answer": "10 and 6"},
    {"question": "A farmer has 17 sheep, and all but 9 run away. How many are left?", "answer": "9"}
]

# Write them to a Python file

In [22]:
with open('/content/best_riddles.py', 'w') as f:
    for i, riddle in enumerate(best_riddles, 1):
        f.write(f"Riddle {i}: {riddle['question']}\n")
        f.write(f"Answer: {riddle['answer']}\n\n")

# Create a Word document and save the 3 best riddles

In [23]:
doc = Document()
doc.add_heading('Best 3 Math Riddles and their Solutions', 0)

for i, riddle in enumerate(best_riddles, 1):
    doc.add_heading(f'Riddle {i}:', level=1)
    doc.add_paragraph(f"Question: {riddle['question']}")
    doc.add_paragraph(f"Answer: {riddle['answer']}")

doc.save('/content/best_riddles.docx')

# Notify the user

In [24]:
print("Best riddles have been saved to 'best_riddles.py' and 'best_riddles.docx'.")

Best riddles have been saved to 'best_riddles.py' and 'best_riddles.docx'.
